In [ ]:
'''
    Notebook to develop/test an algorithm to determine which tiles need to be loaded
'''

import sys
sys.path.append('../src/')
import numpy as np
import matplotlib.pyplot as plt
from utils import *

In [ ]:
#         | 6 | 7 | 8 |
#         -------------
#         | 3 | 4 | 5 |
#         -------------
#         | 0 | 1 | 2 |

def update_tiles(loaded_tiles, curr_tile_id, new_tile_id, header, load_per_dim):
    if new_tile_id == curr_tile_id:
        return
    
    rshift = (new_tile_id % header["n_x_tiles"]) - (curr_tile_id % header["n_x_tiles"])
    ushift = (new_tile_id // header["n_x_tiles"]) - (curr_tile_id // header["n_x_tiles"])
            
    tile_shift = rshift + ushift*load_per_dim
    
    print(f"Rshift: {rshift}, Ushift: {ushift}, tileshift: {tile_shift}")
    
    # Indicates if an index can be shifted
    shift_mask = [0]*(load_per_dim**2)
    # Indicates if an index needs reloading (initialized with reload ALL)
    load_mask = [True]*(load_per_dim**2)
    
    # Get new tile indices
    new_loaded_tiles = [0]*(load_per_dim**2)
    new_tiles(new_loaded_tiles, new_tile_id, load_per_dim, header)
    
    if abs(rshift) >= load_per_dim or abs(ushift) >= load_per_dim:
        # Requires complete reload
        # This case can be skipped if we call readAllTiles after if
        #readAllTiles(load_mask, new_loaded_tiles)
        print("Requiring complete reload")
        pass
    else:
        # Determine which tiles are still visible and where we have to shift them.
        for i in range(0, load_per_dim):
            for j in range(0, load_per_dim):
                #col I, row J
                i_curr = i
                if rshift < 0:
                    i_curr = load_per_dim - i -1
                j_curr = j
                if ushift < 0:
                    j_curr = load_per_dim - j -1
                    
                old_i = i_curr + rshift
                old_j = j_curr + ushift
                if old_i < 0 or old_j < 0 or old_i > (load_per_dim-1) or old_j > (load_per_dim-1):
                    # Tile needs to be loaded
                    print(f"SD->{i_curr+j_curr*(load_per_dim)}")
                    pass
                else:
                    # Tile can be reused
                    load_mask[i_curr+j_curr*(load_per_dim)] = False
                    shift_mask[i_curr+j_curr*(load_per_dim)] = old_i+old_j*(load_per_dim)
                    print(f"{old_i+old_j*(load_per_dim)}->{i_curr+j_curr*(load_per_dim)}")
                
            # We can keep tile with old index i if:
    
                
    print(f"Load mask: {load_mask}")
    print(f"Shift mask: {shift_mask}")
    # mock_Copy(shift_mask, load_mask, rshift, ushift)

# Equivalent to getTileBlock
def new_tiles(loaded_tiles, curr_tile_id, load_per_dim, header):
    tile_ids = [0]*(load_per_dim**2)
    # ID of lower-left tile
    ll_id = curr_tile_id - (load_per_dim//2) - (load_per_dim//2)*header["n_x_tiles"]
    for i in range(0, load_per_dim**2):
        tile_ids[i] = ll_id + (i // load_per_dim)*header["n_x_tiles"] + (i % load_per_dim)
    return tile_ids

In [ ]:
# Get a header
bin_path = "../../data/bin_maps/dach.bin"
header = read_header(bin_path)

# Define current position
curr_pos = [48.136223, 11.594579]

# Define how many tiles per dimension we should load
load_per_dim = 3

# Change to next tile relative to current tile
n_right = 0
n_up = 1



In [ ]:
# Get current tile ID
curr_tile_id = get_tileid_from_latlon(curr_pos[0], curr_pos[1], header)
# Get next tile ID
next_tile_id = curr_tile_id + n_right + n_up*header["n_x_tiles"]

# Get all tile IDs we would need to load initially
loaded_tiles = [0]*(load_per_dim**2)
loaded_tiles = new_tiles(loaded_tiles, curr_tile_id, load_per_dim, header)
update_tiles(loaded_tiles, curr_tile_id, next_tile_id, header, load_per_dim)